<a href="https://colab.research.google.com/github/tkwongspace/Mangrove_Stability/blob/main/Mangrove_Stability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mangrove Stability

An all-in-one script to extract time series of the Normalized Difference Vegetation Index (NDVI) and the Near Infra-Red Vegetation (NIRv) index from Landsat-7 imagery on Google Earth Engine. 

In [ ]:
import time
import math
import json
import geopandas as gpd
import ee

## Supportive Tools

**BRDF Function**

A function to adjust band values with bidirectional reflectance distribution function (BRDF). Codes created by Daniel Wiell & Erik Lindquist from the UNFAO.

Original codes are shared at https://code.earthengine.google.com/3a6761dea6f1bf54b03de1b84dc375c6.

Methods published in Roy DP *et al.* A general method to normalize Landsat reflectance data to narid BRDF adjusted reflectance. *Remote Sensing of Environment*. 2016, 176: 255-271.

In [ ]:
def brdf_correct(image):
    """
    Performs BRDF correction on Landsat image.
    :param image: ee.Image object to be corrected
    :return: ee.Image
    """
        
    # ----- embedded common functions -----
    def x(point):
        return ee.Number(ee.List(point).get(0))
    
    def y(point):
        return ee.Number(ee.List(point).get(1))
    
    def point_between(point1, point2):
        return ee.Geometry.LineString([point1, point2]).centroid().coordinates()
    
    def slope_between(point1, point2):
        return (y(point1).subtract(y(point2))).devide(x(point1).subtract(x(point2)))
    
    def to_line(p1, p2):
        return ee.Geometry.LineString([p1, p2])
    
    def set_image(img, name, to_add, args):
        to_add  = to_image(img, to_add, args)
        img = img.addBands(to_add.rename(name), None, True)
        
    def set_if(img, name, condition, true_value, false_value, args):
        condition = to_image(img, condition, const)
        true_masked = to_image(img, true_value, const) \
            .mask(to_image(img, condition, const))
        false_masked = to_image(img, false_value, const) \
            .mask(invert_mask(to_image(img, condition, const)))
        value = true_masked.unmask(false_masked)
        set_image(img, name, value, const)
    
    def invert_mask(mask):
        return mask.multiply(-1).add(1)
    
    def to_image(img, band, args):
        if isinstance(band, str):
            if '.' in band or ' ' in band or '{' in band:
                band = img.expression(format_expression(band, args), {'i': img})
            else:
                band = img.select(band)
        return ee.Image(band)
    
    def format_expression(s, args):
        all_args = {**const, **args}
        result = s.format(**all_args)
        while '{' in result:
            result  = result.format(**all_args)
        return result
    
    # ----- Embedded main functions -----
    def adjust_bands(img, coefficients_by_band):
        for band_name, coefficients in coefficients_by_band.items():
            apply_c_factor(img, band_name, coefficients)
    
    def angle_prime(img, name, angle, cnt):
        args = {'b/r': 1, 'angle': angle}
        set_image(img, 'tanAnglePrime', 
                  '{b/r} * tan({angle})', args)
        set_if(img, 'tanAnglePrime', 
               'i.tanAnglePrime < 0', 0, 'tanAnglePrime', cnt)
        set_image(img, name, 
                  'atan(i.tanAnglePrime)', cnt)
        
    def apply_c_factor(img, band_name, coefficients):
        brdf(img, 'brdf', 'kvol', 'kgeo', coefficients)
        brdf(img, 'brdf0', 'kvol0', 'kgeo0', coefficients)
        set_image(img, 'cFactor', 'i.brdf0 / i.brdf', coefficients)
        set_image(img, band_name, 
                  '{bandName} * i.cFactor',
                  {**coefficients, 'bandName': 'i.' + band_name})

    def brdf(img, band_name, kvol_band, kgeo_band, coefficients):
        args = {**coefficients, 
                'kvol': '3 * i.' + kvol_band,
                'kgeo': 'i.' + kgeo_band}  # Note the multiplication factor
        set_image(img, band_name, 
                  '{fiso} + {fvol} * {kvol} + {fgeo} * {kvol}', args)

    def cos_phase_angle(img, name, sun_zen, view_zen, relative_sun_view_az, cnt):
        args = {'sunZen': sun_zen, 
                'viewZen': view_zen, 
                'relativeSunViewAz': relative_sun_view_az}
        set_image(img, name,
                  to_image(img,
                           'cos({sunZen}) * cos({viewZen})'
                           ' + sin({sunZen}) * sin({viewZen}) * cos({relativeSunViewAz})',
                           args).clamp(-1, 1), cnt)

    def find_corners(img):
        footprint = ee.Geometry(img.get('system:footprint'))
        bounds = ee.List(footprint.bounds().coordinates().get(0))
        coords = footprint.coordinates()

        xs = coords.map(lambda item: x(item))
        ys = coords.map(lambda item: y(item))

        def find_corner(target_value, values):
            diff = values.map(lambda value: ee.Number(value).subtract(target_value).abs())
            min_value = diff.reduce(ee.Reducer.min())
            idx = diff.indexOf(min_value)
            return coords.get(idx)

        lower_left = find_corner(x(bounds.get(0)), xs)
        lower_right = find_corner(y(bounds.get(1)), ys)
        upper_right = find_corner(x(bounds.get(2)), xs)
        upper_left = find_corner(y(bounds.get(3)), ys)

        return {
            'upperLeft': upper_left,
            'upperRight': upper_right,
            'lowerRight': lower_right,
            'lowerLeft': lower_left
        }
    
    def view_angles(img, cnr):
        max_distance_to_scene_edge = 1000000
        max_satellite_zenith = 7.5
        upper_center = point_between(cnr['upperLeft'], cnr['upperRight'])
        lower_center = point_between(cnr['lowerLeft'], cnr['lowerRight'])
        slope = slope_between(lower_center, upper_center)
        slope_perp = ee.Number(-1).divide(slope)
        set_image(img, 'viewAz', ee.Image(ee.Number(math.pi / 2) \
                                          .subtract(slope_perp.atan())), {})

        left_line = to_line(cnr['upperLeft'], cnr['lowerLeft'])
        right_line = to_line(cnr['upperRight'], cnr['lowerRight'])
        left_distance = ee.FeatureCollection(left_line).distance(max_distance_to_scene_edge)
        right_distance = ee.FeatureCollection(right_line).distance(max_distance_to_scene_edge)
        view_zenith = right_distance.multiply(max_satellite_zenith * 2) \
            .divide(right_distance.add(left_distance)) \
            .subtract(max_satellite_zenith)
        set_image(img, 'viewZen', view_zenith.multiply(math.pi).divide(180), {})
    
    def solar_position(img, cnt):
        date = ee.Date(ee.Number(img.get('system:time_start')))
        seconds_in_hour = 3600

        set_image(img, 'longDeg', ee.Image.pixelLonLat().select('longitude'), {})
        set_image(img, 'latRad',
                  ee.Image.pixelLonLat().select('latitude') \
                  .multiply(cnt['pi']).divide(180), {})
        set_image(img, 'hourGMT', ee.Number(date.getRelative('second', 'day')).divide(seconds_in_hour), {})
        set_image(img, 'jdp', date.getFraction('year'), {})
        set_image(img, 'jdpr', 'i.jdp * 2 * {pi}', cnt)
        set_image(img, 'meanSolarTime', 'i.hourGMT + i.longDeg / 15', cnt)
        set_image(img, 'localSolarDiff', 
                  '(0.000075 + 0.001868 * cos(i.jdpr) - 0.032077 * sin(i.jdpr) ' +
                  '- 0.014615 * cos(2 * i.jdpr) - 0.040849 * sin(2 * i.jdpr)) * 12 * 60 / {pi}', cnt)
        set_image(img, 'trueSolarTime', 
                  'i.meanSolarTime + i.localSolarDiff / 60 - 12', cnt)
        set_image(img, 'angleHour', 'i.trueSolarTime * 15 * {pi} / 180', cnt)
        set_image(img, 'delta',
                  '0.006918 - 0.399912 * cos(i.jdpr) + 0.070257 * sin(i.jdpr) - 0.006758 * cos(2 * i.jdpr) ' +
                  '+ 0.000907 * sin(2 * i.jdpr) - 0.002697 * cos(3 * i.jdpr) + 0.001480 * sin(3 * i.jdpr)', cnt)
        set_image(img, 'cosSunZen', 
                  'sin(i.latRad) * sin(i.delta) + cos(i.latRad) * cos(i.delta) * cos(i.angleHour)',
                  cnt)
        set_image(img, 'sunZen', 'acos(i.cosSunZen)', cnt)
        set_image(img, 'sinSunAzSW',
                  to_image(img, 'cos(i.delta) * sin(i.angleHour) / sin(i.sunZen)', cnt).clamp(-1, 1), cnt)
        set_image(img, 'cosSunAzSW',
                  '(-cos(i.latRad) * sin(i.delta) + sin(i.latRad) * cos(i.delta) * cos(i.angleHour)) / sin(i.sunZen)',
                  cnt)
        set_image(img, 'sunAzSW', 'asin(i.sinSunAzSW)', cnt)
        set_if(img, 'sunAzSW', 'i.cosSunAzSW <= 0', '{pi} - i.sunAzSW', 'sunAzSW', cnt)
        set_if(img, 'sunAzSW', 'i.cosSunAzSW > 0 and i.sinSunAzSW <= 0', '2 * {pi} + i.sunAzSW', 'sunAzSW', cnt)
        set_image(img, 'sunAz', 'i.sunAzSW + {pi}', cnt)
        set_if(img, 'sunAz', 'i.sunAz > 2 * {pi}', 'i.sunAz - 2 * {pi}', 'sunAz', cnt)
    
    def sun_zen_out(img, cnt):
        set_image(img, 'centerLat',
                  ee.Number(
                      ee.Geometry(img.get('system:footprint')). \
                          bounds().centroid(30).coordinates().get(0)) \
                  .multiply(cnt['pi']).divide(180), {})
        set_image(img, 'sunZenOut', 
                  '(31.0076 - 0.1272 * i.centerLat + 0.01187 * pow(i.centerLat, 2) ' +
                  '+ 2.40E-05 * pow(i.centerLat, 3) - 9.48E-07 * pow(i.centerLat, 4) - 1.95E-09 * pow(i.centerLat, 5) ' +
                  '+ 6.15E-11 * pow(i.centerLat, 6)) * {pi}/180', cnt)
        
    def ross_thick(img, band_name, sun_zen, view_zen, relative_sun_view_az, cnt):
        args = {'sunZen': sun_zen, 
                'viewZen': view_zen, 
                'relativeSunViewAz': relative_sun_view_az}
        cos_phase_angle(img, 'cosPhaseAngle',
                        sun_zen, view_zen, relative_sun_view_az, cnt)
        set_image(img, 'phaseAngle', 'acos(i.cosPhaseAngle)', cnt)
        set_image(img, band_name, 
                  '(({pi}/2 - i.phaseAngle) * i.cosPhaseAngle + sin(i.phaseAngle)) ' +
                  '/ (cos({sunZen}) + cos({viewZen})) - {pi}/4', {**args, **cnt})

    def li_thin(img, band_name, sun_zen, view_zen, relative_sun_view_az, cnt):
        args = {'sunZen': sun_zen, 
                'viewZen': view_zen, 
                'relativeSunViewAz': relative_sun_view_az, 'h/b': 2}
        angle_prime(img, 'sunZenPrime', sun_zen, cnt)
        angle_prime(img, 'viewZenPrime', view_zen, cnt)
        cos_phase_angle(img, 'cosPhaseAnglePrime', 
                        'i.sunZenPrime', 'i.viewZenPrime', relative_sun_view_az, cnt)
        set_image(img, 'distance', 
                  'sqrt(pow(tan(i.sunZenPrime), 2) + pow(tan(i.viewZenPrime), 2) ' +
                  '- 2 * tan(i.sunZenPrime) * tan(i.viewZenPrime) * cos({relativeSunViewAz}))', args)
        set_image(img, 'temp', 
                  '1/cos(i.sunZenPrime) + 1/cos(i.viewZenPrime)', cnt)
        set_image(img, 'cosT',
                  to_image(img,
                           '{h/b} * sqrt(pow(i.distance, 2) + pow(tan(i.sunZenPrime) * tan(i.viewZenPrime) '
                           '* sin({relativeSunViewAz}), 2)) ' +
                           '/ i.temp', args).clamp(-1, 1), cnt)
        set_image(img, 't', 'acos(i.cosT)', cnt)
        set_image(img, 'overlap', 
                  '(1/{pi}) * (i.t - sin(i.t) * i.cosT) * (i.temp)', cnt)
        set_if(img, 'overlap', 'i.overlap > 0', 0, 'overlap', cnt)
        set_image(img, band_name,
                  'i.overlap - i.temp + (1/2) * (1 + i.cosPhaseAnglePrime)'
                  ' * (1/cos(i.sunZenPrime)) * (1/cos(i.viewZenPrime))',
                  cnt)
    
    # ----- main process -----
    input_band_names = image.bandNames()    
    const = {'pi': math.pi}
    coefficients_by_bands = {
        'blue': {'fiso': 0.0774, 'fgeo': 0.0079, 'fvol': 0.0372},
        'green': {'fiso': 0.1306, 'fgeo': 0.0178, 'fvol': 0.0580},
        'red': {'fiso': 0.1690, 'fgeo': 0.0227, 'fvol': 0.0574},
        'nir': {'fiso': 0.3093, 'fgeo': 0.0330, 'fvol': 0.1535},
        'swir1': {'fiso': 0.3430, 'fgeo': 0.0453, 'fvol': 0.1154},
        'swir2': {'fiso': 0.2658, 'fgeo': 0.0387, 'fvol': 0.0639}
    }
    
    corners = find_corners(input_band_names)
    
    view_angles(image, corners)
    solar_position(image, const)
    sun_zen_out(image, const)
    set_image(image, 'relativeSunViewAz', 
              'i.sunAz - i.viewAz', const)
    ross_thick(image, 'kvol', 
               'i.sunZen', 'i.viewZen', 'i.relativeSunViewAz', const)
    ross_thick(image, 'kvol0', 
               'i.sunZenOut', 0, 0, const)
    li_thin(image, 'kgeo', 
            'i.sunZen', 'i.viewZen', 'i.relativeSunViewAz', const)
    li_thin(image, 'kgeo0', 
            'i.sunZenOut', 0, 0, const)
    adjust_bands(image, coefficients_by_bands)
    
    return image.select(input_band_names)


**Image Process**

Functions to perform different processes on images.

In [ ]:
# supportive functions
def mask_landsat7_sr(image):
    """
    Creates bit masks for cloud shadow and cirrus from the band QA_PIXEL,
    and applies the mask to retain clear pixels only.

    :param image: ee.Image
    :return: ee.Image
    """
    cloud_shadow_bit_mask = (1 << 3)
    cloud_bit_mask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_bit_mask).eq(0))
    return image.updateMask(mask)

def apply_scaling_offset(image):
    """
    Scales and offsets specified bands in a Landsat 7 image.
    :param image: ee.Image
    :return: ee.Image
    """
    bands_to_modify = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
    scale = ee.Number(2.75e-05)
    offset = ee.Number(-0.2)

    def scale_band(band_name):
        band = image.select(band_name)
        return band.multiply(scale).add(offset).rename(band_name)

    # scale and offset each specified band
    scaled_bands = [scale_band(band_name) for band_name in bands_to_modify]
    scaled_image = ee.ImageCollection(scaled_bands).toBands()
    original_names = ee.List(bands_to_modify)
    renamed_scaled_image = scaled_image.rename(original_names)

    # combine scaled bands with the original bands
    modified_image = image.select(image.bandNames().removeAll(bands_to_modify)) \
        .addBands(renamed_scaled_image)

    return modified_image

def get_ndvi(image):
    """
    Calculates normalized difference vegetation index (NDVI) from Landsat 7 image.
    :param image: ee.Image
    :return: ee.Image
    """
    ndvi = image.expression(
        '(nir - red) / (nir + red)', {
            'nir': image.select('nir'),
            'red': image.select('red')
        }).rename('ndvi').toFloat()
    return image.addBands(ndvi)

def get_nirv(image):
    """
    Calculates Near-Infrared Reflectance of Vegetation (NIRv) from Landsat 7 image.
    :param image: ee.Image
    :return: ee.Image
    """
    nirv = image.expression(
        '((nir - red) / (nir + red)) * nir', {
            'nir': image.select('nir'),
            'red': image.select('red')
        }).rename('nirv').toFloat()
    return image.addBands(nirv)

def get_vi_time_series(feature, image_collection, start_date, end_date, target='ndvi'):
    """
    Calculate the time series of mean pixel-based vegetation index over the given feature,
    using the longitude and latitude of the feature centroid to mark the location.
    :param feature: ee.Feature, the region of interest.
    :param image_collection: ee.ImageCollection, the image collection to map over.
    :param start_date: string, the start date of image, in format 'YYYY-MM-dd'.
    :param end_date: string, the end date of image, in format 'YYYY-MM-dd'.
    :param target: string, ndvi or nirv, default to ndvi.
    :return: ee.FeatureCollection, containing centroid location and VI values in each feature.
    """
    # get centroid location of the given feature
    centroid = feature.geometry().centroid()
    lon = centroid.coordinates().get(0)
    lat = centroid.coordinates().get(1)
    # filter images by date and location, and apply pre-process on images
    ic_filtered = image_collection \
        .filterBounds(feature.geometry()) \
        .filterDate(start_date, end_date) \
        .map(mask_landsat7_sr) \
        .map(apply_scaling_offset) \
        .map(brdf_correct) \
        .map(get_ndvi) \
        .map(get_nirv)

    def calc_mean_vi(image, vi):
        return ee.Image(image).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=30,
            maxPixels=1e9
        ).get(vi)

    def convert_list_to_feature(element):
        return ee.Feature(None, {
            'lon': lon,
            'lat': lat,
            'prop': element
        })

    # create a list of image and extract mean NDVI values from each image
    image_list = ic_filtered.toList(ic_filtered.select(target).size())
    vi_list = [calc_mean_vi(img, target) for img in image_list]
    id_list = ic_filtered.aggregate_array('system:index')
    # join all lists
    values = id_list.zip(vi_list)

    return ee.FeatureCollection([convert_list_to_feature(ele) for ele in values])

## Everything's Ready!

It's time to start the main process!

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='ee-charleshzijian')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load image
full_mangrove_path = r'/content/drive/My Drive/Mangrove/China/ChinaMangrove2020'
export_path = r'/content/drive/My Drive/Mangrove/China/ChinaMangrove2020/subsets'
result_path = r'/content/drive/My Drive/Mangrove/China/ChinaMangrove2020/vi_ts'
# Date range to proceed
start_date = '1999-01-01'
end_date = '2019-12-31'

In [ ]:
# Read shapefile
full_mangrove = gpd.read_file(full_mangrove_path + '/ChinaMangrove2020.shp'). \
    to_crs("epsg:4326")

In [ ]:
# Load the Landsat 7 image collection
ic = ee.ImageCollection('LANDSAT/LE07/C02/T2_L2') \
    .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'],
            ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'QA_PIXEL'])

In [ ]:
# split and calculate indices on every 40 features each time
shp_idx = 0
feature_length = 80  # len(full_mangrove)
for i in range(0, feature_length, 40):
    shp_idx += 1
    print(f">> Now on #{shp_idx}..")
    # -- get features ready
    # slice the geo-dataframe
    gdf = full_mangrove.iloc[i:i+40]
    # first export the sliced geo-dataframe into a new shapefile
    export_slice = f"ChinaMangrove_part{shp_idx}.shp"
    gdf.to_file(f"{export_path}/{export_slice}", driver='ESRI Shapefile')
    # -- calculate indices based on features
    for vi in ['ndvi', 'nirv']:
        # convert the geo-dataframe to a list of dictionaries
        features = json.loads(gdf.to_json())["features"]
        # create a list of Earth Engine features
        ee_features = []
        for feature in features:
            # extract geometry and properties
            geometry = ee.Geometry.MultiPolygon(feature['geometry']['coordinates'])
            properties = feature['properties']
            # create an Earth Engine feature
            ee_feature = ee.Feature(geometry, properties)
            # append to list
            ee_features.append(ee_feature)
        # create the feature collection
        features = ee.FeatureCollection(ee_features)
        
        # calculate mean vegetation indices for each feature
        result = features.map(lambda f: get_vi_time_series(
            feature = f,
            image_collection = ic,
            start_date = start_date,
            end_date = end_date,
            target = vi
        ))
        
        # export the result to a csv file
        task = ee.batch.Export.table.toDrive(
            collection = result,
            description = f'Mean_{vi}_{shp_idx}',
            folder = export_path,
            file_name = 'CSV'
        )
        task.start()
        print(f"-- Task submitted at {time.strftime('%Y-%m-%d %H:%M:%S', time.time())}.")
        
        # Wait for a while before processing the next loop to avoid rate limits
        time.sleep(30)

print('>> ALL FEATURES PROCESSED.')